In [ ]:
import os
import pandas as pd
from langchain_core.documents import Document
from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
from langchain_community.document_loaders import CSVLoader
load_dotenv(override=True)


# 1) Load & sanitize
df = pd.read_csv("joblist_clean_for_rag.csv")
df = df.fillna("").reset_index(drop=False).rename(columns={"index": "doc_id"})
df["doc_id"] = df["doc_id"].astype(str)

def row_to_doc(row):
    content = (
        f"Title: {row['title']}\n"
        f"Company: {row['company']}\n"
        f"Location: {row['location']}\n"
        f"Remote: {row['remote']}\n"
        f"Department: {row['department']}\n"
        f"Description:\n{row['description']}"
    ).strip()
    metadata = {
        "doc_id": row["doc_id"],
        "title": row["title"],
        "company": row["company"],
        "location": row["location"],
        "remote": row["remote"],
        "department": row["department"],
    }
    return Document(page_content=content, metadata=metadata)

base_docs = [row_to_doc(r) for _, r in df.iterrows()]
print(f"Loaded {len(base_docs)} base docs")

# 2) Chunk (so each embed is small)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,     # ~800–1200 chars is usually fine
    chunk_overlap=150,   # small overlap for context continuity
    separators=["\n\n", "\n", " ", ""],
)

chunked_docs = []
for d in base_docs:
    for i, ch in enumerate(splitter.split_text(d.page_content)):
        md = dict(d.metadata)
        md["chunk"] = i
        chunked_docs.append(Document(page_content=ch, metadata=md))

print(f"Prepared {len(chunked_docs)} chunks")

# 3) Embeddings (use the small model to cut cost/size)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 4) Chroma store
vector_store = Chroma(
    collection_name="jobs_rag",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

# 5) Batch add to avoid per-request token overflow
BATCH_SIZE = 100  # adjust if you still hit limits; smaller => safer
ids = []
for i in range(0, len(chunked_docs), BATCH_SIZE):
    batch = chunked_docs[i:i+BATCH_SIZE]
    ids.extend(vector_store.add_documents(batch))
    # optional: print progress
    # print(f"Indexed {i+len(batch)}/{len(chunked_docs)}")

print("Example IDs:", ids[:3])
print("Done indexing.")


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: './chroma_langchain_db\\990ed5fb-d211-4156-a963-6add85a19cfa\\data_level0.bin'

In [28]:
# 6) Retrieval example
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

query = "senior Python backend engineer with experience in low-latency trading"
results = retriever.invoke(query)
for r in results:
    print("-----")
    print(r.metadata.get("doc_id"), "|", r.metadata.get("title"), "|", r.metadata.get("company"), "|", r.metadata.get("location"))
    print(r.page_content[:400], "...")


-----
373 | Staff Backend Engineer (Python), AI Engineering: Agent Foundations  | gitlab | Remote, EMEA
Multiple years of experience building Python Web Services in a multi-service environment ( e.g. FastAPI, Starlette)
Proven ability to architect and lead complex technical projects from concept to production across distributed systems
Strong experience with AI/ML technologies, including integration with large language models and understanding of prompt engineering
Deep expertise in performance opti ...
-----
None | Staff Backend Engineer (Python), AI Engineering: Agent Foundations  | gitlab | Remote, EMEA
Multiple years of experience building Python Web Services in a multi-service environment ( e.g. FastAPI, Starlette)
Proven ability to architect and lead complex technical projects from concept to production across distributed systems
Strong experience with AI/ML technologies, including integration with large language models and understanding of prompt engineering
Deep expertise in pe

In [23]:
# Index df by doc_id for fast lookup
df_by_id = df.set_index("doc_id", drop=False)

def format_full_row(row):
    return (
        f"Title: {row['title']}\n"
        f"Company: {row['company']}\n"
        f"Location: {row['location']}\n"
        f"Remote: {row['remote']}\n"
        f"Department: {row['department']}\n"
        f"Description:\n{row['description']}"
    ).strip()

def retrieve_full_jobs_by_id(query, k=5):
    chunks = retriever.invoke(query)
    seen = set()
    full = []
    for ch in chunks:
        did = ch.metadata["doc_id"]
        if did in seen:
            continue
        seen.add(did)
        row = df_by_id.loc[did]
        full.append(format_full_row(row))
        if len(full) >= k:
            break
    return full

# Example
full_jobs = retrieve_full_jobs_by_id("oracle xstore pos rollout lead", k=3)
print(full_jobs[0][:1000])


KeyError: 'doc_id'

In [25]:
for _, r in df.iterrows():
    print(r)

doc_id                                                             0
title                                                   Academy Lead
company                                                        adyen
location                                                   Amsterdam
remote                                                         False
department                                             Adyen Academy
description        This is Adyen \n Adyen provides payments, data...
salary_currency                                                     
salary_min                                                          
salary_max                                                          
Name: 0, dtype: object
doc_id                                                             1
title                                                Account Manager
company                                                        adyen
location                                                      Berlin
remote     